In [1]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
from joblib import Parallel, delayed
from IPython.display import display, HTML
from matplotlib import animation, rc
rc('animation', html='jshtml')
# for DA
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as Data
import ast #?
from fastprogress.fastprogress import master_bar, progress_bar #?

In [2]:
def remove_file(old_path, new_path):
    filelist = os.listdir(old_path)
    cnt = 0
    for file in filelist:
        src = os.path.join(old_path, file)
        dst = os.path.join(new_path, file)
        shutil.copyfile(src, dst)
        cnt += 1
        if cnt == 4700:
            break


In [3]:
df = pd.read_csv('great-barrier-reef/train.csv')
df['old_image_path'] = f'great-barrier-reef/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['image_path']  = f'images/all_img/'+df.image_id+'.jpg' # '/kaggle/working/images'
df['label_path']  = f'images/all_label/'+df.image_id+'.txt' # '/kaggle/working/labels'
df['annotations'] = df['annotations'].progress_apply(eval)

  0%|          | 0/23501 [00:00<?, ?it/s]

In [4]:
def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return
image_paths = df.old_image_path.tolist()
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _, row in tqdm(df.iterrows(), total=len(df)))

remove_file('images/all_img', 'images/test')

  0%|          | 0/23501 [00:00<?, ?it/s]

In [5]:
for i in range(0, 18800):
    img = cv2.imread(df.iloc[i]['image_path'])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    empty = []
    if df.iloc[i]['annotations'] != empty:
        y0 = df.iloc[i]['annotations'][0]['y']
        x0 = df.iloc[i]['annotations'][0]['x']
        y1 = y0 + df.iloc[i]['annotations'][0]['height']
        x1 = x0 + df.iloc[i]['annotations'][0]['width']
        cropped = img_rgb[y0:y1, x0:x1]
        crop_neg = img_rgb[0: 100, 0:100]
#         print(df.iloc[i]['image_path'])
        if cropped.any(): 
            cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
            pos_name = "images/train/pos/" + df.iloc[i]['image_path'][15:]
            cv2.imwrite(pos_name, cropped)
        if crop_neg.any():
            crop_neg = cv2.cvtColor(crop_neg, cv2.COLOR_BGR2RGB)
            neg_name = "images/train/neg/" + df.iloc[i]['image_path'][15:]
            cv2.imwrite(neg_name, crop_neg)

In [6]:
leng, tmp = df.shape
for j in range(18801, leng):
#     print(df.iloc[i])
    img_ = cv2.imread(df.iloc[j]['image_path'])
    img_name = "images/test/" + df.iloc[j]['image_path'][15:]
    cv2.imwrite(img_name, img_)